In [1]:
# please feel free to install any additional packages you need, use
# !pip install package_name

import pandas as pd

air_traffic_data_train = pd.read_csv('/Users/sedovandrey/Documents/AI coding experiment/hackerrank_input/task_1/data_input/air_traffic_pax_stat_train.csv')
air_traffic_data_validation = pd.read_csv('/Users/sedovandrey/Documents/AI coding experiment/hackerrank_input/task_1/data_input/air_traffic_pax_stat_test.csv')

air_traffic_data_validation.head()

air_traffic_data_train.head()

# Context Features

cons_price = pd.read_csv('/Users/sedovandrey/Documents/AI coding experiment/hackerrank_input/task_1/data_input/CONSUMER_PRICE_INDEX.csv')
sf_pop = pd.read_csv('/Users/sedovandrey/Documents/AI coding experiment/hackerrank_input/task_1/data_input/SF_POPULATION.csv')
gdp = pd.read_csv('/Users/sedovandrey/Documents/AI coding experiment/hackerrank_input/task_1/data_input/US_GDP_PER_CAPITA.csv')
wti_prices = pd.read_csv('/Users/sedovandrey/Documents/AI coding experiment/hackerrank_input/task_1/data_input/WTI_PRICES.csv')

# Merge context features
def add_context_feats(air_traffic_data_train):
    comb = pd.merge(cons_price, sf_pop, on='DATE', how='outer')
    comb = pd.merge(comb, gdp, on='DATE', how='outer')
    comb = pd.merge(comb, wti_prices, on='DATE', how='outer')

    comb = comb.rename(columns={"DATE": "Activity Period"})

    return pd.merge(air_traffic_data_train, comb,
                    on='Activity Period', how='left')

test_data = add_context_feats(air_traffic_data_validation)
data = add_context_feats(air_traffic_data_train)

data.info()

import numpy as np
# Replace missing vlaues with out-of-scope value
data = data.replace(np.nan, -999)
test_data = test_data.replace(np.nan, -999)
data.isnull().sum()

exog = data.columns[1:]
exog_test = test_data.columns[1:]

# Out of time for this
#data = pd.get_dummies(exog)
#test_data = pd.get_dummies(exog_test)
#data.columns

# ARIMA Model

!pip install pmdarima



In [ ]:
import pmdarima as pm

SARIMAX_model = pm.auto_arima(data[['Passenger Count']], exogenous=data[['CPIAUCSL', 'CASANF0POP', 'A939RX0Q048SBEA', 'MCOILWTICO' ]],
                           start_p=1, start_q=1,
                           test='adf',
                           max_p=3, max_q=3, m=24,
                           start_P=0, seasonal=True,
                           d=None, D=1,
                           trace=False,
                           error_action='ignore',
                           suppress_warnings=True,
                           stepwise=True)

SARIMAX_model.summary()

fitted, confint = SARIMAX_model.predict(n_periods=24,
                                            return_conf_int=True,
                                            exogenous=test_data[exog_test])
fitted = fitted.tolist()
fitted = [int(x) for x in fitted]
air_traffic_data_validation['Passenger Count'] = fitted
air_traffic_data_validation

# Save results for the scoring

result = air_traffic_data_validation
result[['Origin Airport', 'Destination Airport', 'Activity Period', 'Passenger Count']].to_csv('/Users/sedovandrey/Documents/AI coding experiment/hackerrank_input/task_1/evaluation', index=False)

,Activity Period,Origin Airport,Destination Airport,Passenger Count
0,2019-01-01,SFO,CDG,10000
1,2019-02-01,SFO,CDG,10000
2,2019-03-01,SFO,CDG,10000
3,2019-04-01,SFO,CDG,10000
4,2019-05-01,SFO,CDG,10000
5,2019-06-01,SFO,CDG,10000
6,2019-07-01,SFO,CDG,10000
7,2019-08-01,SFO,CDG,10000
8,2019-09-01,SFO,CDG,10000
9,2019-10-01,SFO,CDG,10000
